# PURSUE Python for HEP: Dimuons

* We will now once again reconstruct the mass of the Z using dimuons, but we will take a more careful approach now using the new things we have learned. We will first use a small sample of our data before using a larger one to have increased statistics.

In [ ]:
# Import all neccesary libraries
import numpy as np
import uproot
import awkward as ak

import hist
import matplotlib.pyplot as plt
import mplhep as hep
hep.style.use("CMS")

In [ ]:
file = uproot.open(
    "root://eospublic.cern.ch//eos/opendata/cms/derived-data/AOD2NanoAODOutreachTool/Run2012BC_DoubleMuParked_Muons.root"
)
tree = file["Events"]

In [ ]:
muons = tree.arrays(entry_stop=10000)

# For convenience, lets rename the branches
muons = ak.zip(
    {
        "pt": muons["Muon_pt"],
        "eta": muons["Muon_eta"],
        "phi": muons["Muon_phi"],
        "charge": muons["Muon_charge"]
    }
)

muons

* The process of interest for now is the decay of Z into two leptons, the two leptons being muons.

<div style="text-align: center;">
  <img src="./assets/Zdimuon.png" alt="roottree" style="width: 400px"/>
</div>

* Firsltly, we need to find all possible unique combination of two muons per event. For this we use `ak.combinations()`. One of the convenient things with this function is that if there is less than 2 muons in an event, and thus its impossible to make a combination of two muons, the array of pairs for that event is empty (i.e. its "filtered" out).

In [ ]:
pairs = ak.combinations(muons, 2)
pairs

* Now we separate each pair of muons into their own arrays. This will allows us to do computations with these in a simple way.

In [ ]:
mu1, mu2 = ak.unzip(pairs)

* We can now use the mass formula we used before to compute the dimuon mass.
$$
m_{\mu\mu} = \sqrt{
    2p_{T,0} p_{T,1} * \left(\cosh(\eta_0 - \eta_1) - \cos(\phi_0- \phi_1)\right)
}
$$

In [ ]:
m_dimuon = np.sqrt(
    2 * mu1["pt"] * mu2["pt"] * (np.cosh(mu1["eta"] - mu2["eta"]) - np.cos(mu1["phi"] - mu2["phi"]))
)
m_dimuon

* We now want to plot these mass values in a histogram. However, you can't plot a jagged array directly. You must first use a function such as `ak.ravel()` to "unravel" the array into a flat array. We then pass that into a histogram object and plot it using Matplotlib.

In [ ]:
fig, ax = plt.subplots()

hist.Hist(
    hist.axis.Regular(120, 0, 120, label="mass [GeV]")
).fill(ak.ravel(m_dimuon)).plot1d(ax=ax, histtype="step", color="red", linewidth=0.75, label="$m_{\mu\mu}$")

ax.set_title("Dimuon Mass")
ax.set_ylabel("Count")
ax.set_xlabel("Mass (GeV)")
ax.legend()

plt.show()

* This seems to have made things worse! Now the Z peak is very small compared to before. We will need to fix this.
* Another thing we can try it plotting the maximum dimuon mass for each event. Doing this can help us in the following ways
  * Background reduction: background tends to have lower invariant mass compared to the Z boson
  * Highest dimuon mass is more likely to have come from the direct decay of a heavy particle like the Z rather than from lower mass background processes
  * Helps reduce contribution of random low-mass pairs that are more likely to be background, increasing StoN ratio.

In [ ]:
max_m_dimuon = ak.max(m_dimuon, axis=1)
# This next step removes any `None` values that arrise in events for which there are zero entries in m_dimuon
max_m_dimuon = ak.flatten(max_m_dimuon, axis=0)
max_m_dimuon

In [ ]:
fig, ax = plt.subplots()

hist.Hist(
    hist.axis.Regular(120, 0, 120, label="mass [GeV]")
).fill(max_m_dimuon).plot1d(ax=ax, histtype="step", color="red", linewidth=0.75, label="$m_{\mu\mu}$")

ax.set_title("Dimuon Mass")
ax.set_ylabel("Count")
ax.set_xlabel("Mass (GeV)")
ax.legend()

plt.show()

**Exercises**:
1. Select pairs of muons with opposite charges.
2. Plot the one mass candidate per event that is strictly closest to the Z mass. Use `zmass = 91`. Alternatively, use the value obtained from the Particle library.

In [ ]:
# Answer 1

In [ ]:
# Answer 2
import particle, hepunits
zmass = particle.Particle.findall("Z0")[0].mass / hepunits.GeV